In [1]:
import pandas as pd 

In [6]:
#todo o processo de leitura da tabela com todas as transações do robôs e o agrupamento de 
#apenas as movimentações encerradas, ou seja, que foram compradas e vendidas e vice-versa. 
def ordem_com_negociacao_fechada(position_ID,dados):
    #deal_type = 0  é compra
    aux = dados[dados['order_ticket']== position_ID]
    
    if aux.empty == False:
        aux = dados[(dados['position_ID'] == position_ID)]
        volume_compra = aux[(aux['deal_type']==0)]['volume'].sum()
        volume_venda = aux[(aux['deal_type']==1)]['volume'].sum()
        if volume_compra == volume_venda:
            #print("volume de compra : ",volume_compra, " volume venda",volume_venda,"\n")
            return True
        else:
            return False
    
    else:
        #print('Não tem order_ticket como início. O número é ',position_ID,"\n")
        return False

def tabela_com_ordens_fechadas():
    dados = ler_csv_para_dataframe()
    position_ID_unique = dados['position_ID']
    position_ID_unique =  position_ID_unique.drop_duplicates()
    tabela =  criar_tabela_vazia()
 
    for i in position_ID_unique:
        if ordem_com_negociacao_fechada( i, dados) == True :
            ordem = criar_tabela_vazia()
            aux = dados[(dados['position_ID']== i)]
            volume_compra = aux[(aux['deal_type']==0)]['volume'].sum()

            ordem_abertura = dados[(dados['order_ticket']==i)]
            
            ordem_ultima = dados[(dados['position_ID']==i)]
            ordem_ultima = ordem_ultima.tail(1)
            ordem['symbol'] = ordem_abertura['symbol'] 
            ordem['ticket'] = ordem_abertura['order_ticket'].values
            ordem['order_magic'] = ordem_abertura['order_magic'].values
            ordem['deal_type_open'] = ordem_abertura['deal_type']
            ordem['datetime_open'] = ordem_abertura['transaction_time']
            ordem['price_open'] = ordem_abertura['price']
            ordem['profit'] = dados[(dados['position_ID']==i)]['profit'].sum()
            ordem['volume'] = (volume_compra*2)
            ordem['deal_type_closed'] = ordem_ultima['deal_type'].values
            ordem['datetime_closed'] = ordem_ultima['transaction_time'].values
            ordem['price_closed'] = ordem_ultima['price'].values
            #tabela = add_ordem_na_tabela(ordem,tabela)
            tabela = tabela.append(ordem,ignore_index=True)
    #print(tabela)
    return tabela

def csv_com_ordens_fechadas():  
    dados = ler_csv_para_dataframe()
    position_ID_unique = dados['position_ID']
    position_ID_unique =  position_ID_unique.drop_duplicates()
    tabela = dados
    rejeitados = []
    aceitos = []
    for i in position_ID_unique:
        if ordem_com_negociacao_fechada( i, dados)== False:
            #print("FOI FALSO  ",i)
            ordem_rejeitada = dados[(dados['position_ID']==i)].index
            dados.drop(ordem_rejeitada,inplace=True)
            rejeitados.append(i) 
        aceitos.append(i)
    return tabela

#order_ticket == ticket
def criar_tabela_vazia():
    nome_coluna = {'symbol':[] ,'ticket':[] ,'order_magic':[] ,'deal_type_open':[],'datetime_open':[],'price_open':[],'deal_type_closed':[],'datetime_closed':[],'price_closed':[],'profit':[],'volume':[]} 
    tabela = pd.DataFrame(data=nome_coluna)
    return tabela

def ordem_com_negociacao_fechada(position_ID,dados):
    #deal_type = 0  é compra
    aux = dados[dados['order_ticket']== position_ID]
    
    if aux.empty == False:
        aux = dados[(dados['position_ID'] == position_ID)]
        volume_compra = aux[(aux['deal_type']==0)]['volume'].sum()
        volume_venda = aux[(aux['deal_type']==1)]['volume'].sum()
        if volume_compra == volume_venda:
            #print("volume de compra : ",volume_compra, " volume venda",volume_venda,"\n")
            return True
        else:
            return False
    
    else:
        #print('Não tem order_ticket como início. O número é ',position_ID,"\n")
        return False

def ler_csv_para_dataframe():
    caminho_base = 'tabelas/ELET3_deals_com_lucro_2019.csv'
    #caminho = 'BBDC4_deals_com_lucro_2019-versao4.csv'
    dados_read = pd.read_table(caminho_base, encoding = 'utf-16',sep=',') 
    dados=pd.DataFrame(dados_read)
    #print(dados)
    return dados

def inicio():
    return csv_com_ordens_fechadas()


In [7]:
#exibindo todas as transações válidas dentro dentro do período registrado, ou seja, as operações
#que tem o seu order_ticket válido e que marca o início e o fim de uma transação completa.
tabela_geral_fechada = inicio()
tabela_geral_fechada

,symbol,deal_ticket,volume,transaction_time,price,order_ticket,deal_type,position_ID,profit,order_magic,balance
0,PETR4,192685913,100,2019.12.02 15:37:52,29.21,259182895,1,259182895,0,12003,95656
3,B3SA3,200641653,100,2019.12.27 14:40:08,44.75,269242890,1,269242890,0,27893,95656
4,BBDC4,200641917,200,2019.12.27 14:40:43,36.38,269243195,0,269243195,0,27893,95656
5,BBDC4,203223105,200,2020.01.07 10:40:18,36.48,272494646,1,269243195,20,27893,95656
6,BBDC4,203712957,200,2020.01.07 17:44:23,36.07,273060821,1,273060821,0,27893,95656
7,BBDC4,204107441,200,2020.01.08 13:44:45,36.02,273544680,0,273060821,10,27893,95656
8,BBDC4,204136504,200,2020.01.08 14:04:30,36.07,273580723,1,273580723,0,27893,95656
9,BBDC4,204996776,200,2020.01.10 10:03:07,35.10,274640549,0,273580723,194,27893,95656
12,CIEL3,212524330,1000,2020.01.29 10:38:44,7.26,284205487,1,284205487,0,27893,95656
13,PETR4,215758234,100,2020.02.05 10:08:59,29.09,288440946,0,259182895,12,12003,95656


In [8]:
#todas as movimentações encerradas e que podem computadas por ter o seu ciclo encerrado (order_ticket)
tabela_ordem = tabela_com_ordens_fechadas()
tabela_ordem

,symbol,ticket,order_magic,deal_type_open,datetime_open,price_open,deal_type_closed,datetime_closed,price_closed,profit,volume
0,PETR4,259182895.0,12003.0,1.0,2019.12.02 15:37:52,29.21,0.0,2020.02.05 10:08:59,29.09,12.0,200.0
1,B3SA3,269242890.0,27893.0,1.0,2019.12.27 14:40:08,44.75,0.0,2020.03.19 15:21:34,31.63,1312.0,200.0
2,BBDC4,269243195.0,27893.0,0.0,2019.12.27 14:40:43,36.38,1.0,2020.01.07 10:40:18,36.48,20.0,400.0
3,BBDC4,273060821.0,27893.0,1.0,2020.01.07 17:44:23,36.07,0.0,2020.01.08 13:44:45,36.02,10.0,400.0
4,BBDC4,273580723.0,27893.0,1.0,2020.01.08 14:04:30,36.07,0.0,2020.01.10 10:03:07,35.10,194.0,400.0
5,CIEL3,284205487.0,27893.0,1.0,2020.01.29 10:38:44,7.26,0.0,2020.03.10 16:51:40,6.45,810.0,2000.0
